In [ ]:
!pip install modelbit
!pip install numpy --upgrade
!pip install --upgrade --force-reinstall diffusers transformers accelerate safetensors huggingface_hub

In [ ]:
import modelbit
modelbit.login()

In [ ]:
!git clone https://github.com/replicate/cog-sdxl cog_sdxl

In [ ]:
import modelbit as mb
import torch
from diffusers import DiffusionPipeline
from huggingface_hub import hf_hub_download
import base64
import io

def generate_image(prompt: str) -> str:
    """
    Generate an emoji-style image using Stable Diffusion XL with emoji LoRA
    Args:
        prompt: Text description of the image to generate
    Returns:
        Base64 encoded string of the generated image
    """
    from content.cog_sdxl.dataset_and_utils import TokenEmbeddingsHandler

    # Initialize the pipeline
    pipe = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16"
    )

    if torch.cuda.is_available():
        pipe = pipe.to("cuda")

    # Load LoRA weights
    pipe.load_lora_weights("fofr/sdxl-emoji", weight_name="lora.safetensors")

    # Setup text encoders and tokenizers
    text_encoders = [pipe.text_encoder, pipe.text_encoder_2]
    tokenizers = [pipe.tokenizer, pipe.tokenizer_2]

    # Load embeddings
    embedding_path = hf_hub_download(
        repo_id="fofr/sdxl-emoji",
        filename="embeddings.pti",
        repo_type="model"
    )
    embhandler = TokenEmbeddingsHandler(text_encoders, tokenizers)
    embhandler.load_embeddings(embedding_path)

    # Add emoji tokens if not present in prompt
    if "<s0><s1>" not in prompt:
        prompt = f"A <s0><s1> emoji of {prompt}"

    # Generate image
    image = pipe(
        prompt,
        cross_attention_kwargs={"scale": 0.8}
    ).images[0]

    # Convert PIL image to base64 string
    buffered = io.BytesIO()
    image.save(buffered, format="WEBP", optimize=True, quality=85)
    img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')

    return img_str

# Get the directory containing cog_sdxl
import os

# Get all Python files in the cog_sdxl directory
extra_files = []
for root, _, files in os.walk("/content/cog_sdxl"):
    for file in files:
        if file.endswith('.py'):
            file_path = os.path.join(root, file)
            extra_files.append(file_path)

# Deploy to ModelBit with extra files
mb.deploy(
    generate_image,
    python_version="3.11",
    python_packages=[
        "diffusers==0.32.2",
        "transformers==4.50.2",
        "torch==2.1.2",
        "accelerate==1.5.2",
        "safetensors==0.5.3",
        "huggingface-hub==0.29.3",
        "numpy==1.26.0",
        "peft==0.15.0"
    ],
    extra_files=extra_files,  # Include all Python files from cog_sdxl
    require_gpu=True,
    name="emoji_generator"
)